In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17756248206299873567
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22692233216
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16317231569479682687
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [3]:
import tensorflow as tf
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

resnet_model = ResNet50(input_shape=(224,224,3), include_top=False)
resnet_model.trainable = True

model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [4]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_gen = ImageDataGenerator(rotation_range=20, 
                               width_shift_range=0.2, height_shift_range=0.2,
                               horizontal_flip=True)
train_data = train_gen.flow_from_directory('./glaucoma/train', target_size=(224,224),
                                           batch_size=32, class_mode='sparse')

Found 1394 images belonging to 3 classes.


In [7]:
test_gen = ImageDataGenerator()
test_data = test_gen.flow_from_directory('./glaucoma/test', target_size=(224,224), 
                                         batch_size=32, class_mode='sparse')

Found 150 images belonging to 3 classes.


In [8]:
model.fit(train_data, validation_data=test_data, epochs=20)

Epoch 1/20
44/44 [==============================] - 20s 258ms/step - loss: 8.1276 - accuracy: 0.6585 - val_loss: 33.5835 - val_accuracy: 0.4133
Epoch 2/20
44/44 [==============================] - 11s 241ms/step - loss: 0.6448 - accuracy: 0.7346 - val_loss: 2.6306 - val_accuracy: 0.4333
Epoch 3/20
44/44 [==============================] - 10s 236ms/step - loss: 0.7245 - accuracy: 0.7181 - val_loss: 45.8079 - val_accuracy: 0.5267
Epoch 4/20
44/44 [==============================] - 10s 235ms/step - loss: 1.1068 - accuracy: 0.7195 - val_loss: 1.1091 - val_accuracy: 0.5733
Epoch 5/20
44/44 [==============================] - 10s 234ms/step - loss: 0.8635 - accuracy: 0.7037 - val_loss: 85.9133 - val_accuracy: 0.6200
Epoch 6/20
44/44 [==============================] - 10s 234ms/step - loss: 0.6168 - accuracy: 0.7245 - val_loss: 7.1434 - val_accuracy: 0.6200
Epoch 7/20
44/44 [==============================] - 10s 235ms/step - loss: 0.5838 - accuracy: 0.7432 - val_loss: 0.7523 - val_accuracy: 0.6

In [9]:
model.save('glaucoma_model.keras')

In [10]:
from tensorflow.keras.models import load_model
model2 = load_model('glaucoma_model.keras')
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [11]:
from tensorflow.keras.preprocessing import image
img = image.load_img('./glaucoma/test/advanced_glaucoma/1.png', target_size=(224,224))
x = image.img_to_array(img).reshape(-1, 224, 224, 3)

In [12]:
pred = model2.predict(x)
print(pred)

1/1 [==============================] - 1s 713ms/step
[[9.9769753e-01 2.3021083e-03 3.2998983e-07]]


In [13]:
import numpy as np
print(np.argmax(pred, axis=1))

[0]


In [14]:
print(test_data.classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [15]:
y_preds = model2.predict(test_data)
y_pred = np.argmax(y_preds, axis=1)

5/5 [==============================] - 1s 113ms/step


In [16]:
import pandas as pd
pd.crosstab(test_data.classes, y_pred)

col_0,0,1,2
row_0,,,
0,26,4,16
1,8,3,15
2,34,12,32


In [17]:
model2.evaluate(test_data)

5/5 [==============================] - 1s 67ms/step - loss: 0.7775 - accuracy: 0.6800


[0.7775223851203918, 0.6800000071525574]